In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [10]:
# 8.1 Найдите все повторяющиеся строки в данных и удалите их
dupl_columns = list(diabetes.columns)

mask = diabetes.duplicated(subset=dupl_columns)
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}')

diabetes = diabetes.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {diabetes.shape[0]}')

Число найденных дубликатов: 0
Результирующее число записей: 768


In [11]:
# 8.2 найдите все неинформативные признаки в данных и избавьтесь от них
# удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны
low_information_cols = [] 

for col in diabetes.columns:
    top_freq = diabetes[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes = diabetes.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {diabetes.shape[1]}')

diabetes.info()

Gender: 100.0% одинаковых значений
Результирующее число признаков: 9
<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 60.0 KB


In [12]:
# 8.3 Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска
diabetes['Glucose'] = diabetes['Glucose'].apply(lambda x: x if x != 0 else np.nan)
diabetes['BloodPressure'] = diabetes['BloodPressure'].apply(lambda x: x if x != 0 else np.nan)
diabetes['SkinThickness'] = diabetes['SkinThickness'].apply(lambda x: x if x != 0 else np.nan)
diabetes['Insulin'] = diabetes['Insulin'].apply(lambda x: x if x != 0 else np.nan)
diabetes['BMI'] = diabetes['BMI'].apply(lambda x: x if x != 0 else np.nan)
cols_null_percent = diabetes.isnull().mean()
display(round(cols_null_percent,2))

Pregnancies                 0.00
Glucose                     0.01
BloodPressure               0.05
SkinThickness               0.30
Insulin                     0.49
BMI                         0.01
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [13]:
# 8.4 далите из данных признаки, где число пропусков составляет более 30 %
thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(thresh=thresh, axis=1)
display(diabetes.shape[1])

8

In [14]:
# 8.5 Удалите из данных только те строки, в которых содержится более двух пропусков одновременно
m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh=m-2, axis=0)
print(diabetes.shape[0])

761


In [15]:
# 8.6 замените пропуски на медиану
values = {
    'BloodPressure': diabetes['BloodPressure'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'Glucose': diabetes['Glucose'].median(),
    'BMI': diabetes['BMI'].median(),
}
diabetes = diabetes.fillna(values)
diabetes.isnull().mean()
diabetes['SkinThickness'].mean().round(1)


29.1

In [16]:
# 8.7 Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [17]:
# 8.8 Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness
def outliers_z_score(data, feature, log_scale=False): # log_scale True, то будем логарифмировать рассматриваемый признак
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(diabetes, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [22]:
# 8.9 Найдите сначала число выбросов в признаке DiabetesPedigreeFunction
# с помощью классического метода межквартильного размаха(29)
def outliers_iqr(data, feature,log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, _ = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_log.shape[0])

29
